Setup parameters for weight adaption session (results are only optimal for the used dct_dim)

In [1]:
#There are the relevant hyper-parameters for weight adaption session (must be the same as in create_training_paths):
dct_dim = 32
min_img_dims = 128
#Experimental switches:
use_pil_rz = False #uses PIL LANCZOS for downsampling instead of opencv INTER_AREA  
num_threads = 16 #use multi-core procedures with this many cores where possible

In [3]:
import numpy as np
#see create_training_paths for creation of the precalculated dcts; 
#due to copyright these may present an issue when uploaded/clash with the open license -> you have to repeat the script yourself
precalc_dcts = np.load('ordered_dct_balanced.npz')

In [4]:
orig_dct, all_bu = precalc_dcts['dcts'], precalc_dcts['paths']

In [5]:
from build.naphash_cpp import naphash as nhcpp, rot_inv_type
orig_dct1 = orig_dct[:]
init_weights = np.ones(324,np.float32)
h_crop0 = nhcpp(dct_dim=dct_dim, rot_inv_mode=rot_inv_type.none, apply_center_crop=False, is_rgb=False)
h_crop0.get_norm(init_weights)
print(init_weights)

[ 421.  256.  317.  352.  642.  832.  703.  590.  657.  603.  992. 1051.
  659.  744. 1059.  909. 1159. 1353. 1180.  960. 1061. 1043. 1602. 1560.
 1176. 1109. 1284. 1556. 1484. 1347. 1695. 1823. 1616. 1207. 1340. 1448.
 1724. 1842. 1474. 1495. 2062. 2096. 1617. 1533. 1908. 1751. 1720. 2079.
 1893. 1568. 2179. 2364. 2036. 1744. 1926. 1949. 2126. 2334. 1929. 1946.
 2590. 2655. 2017. 1917. 2163. 2430. 2355. 2155. 2170. 2593. 2349. 2162.
 2730. 2964. 2467. 2151. 2205. 2306. 2588. 2721. 2278. 2469. 2564. 2902.
 2307. 2334. 3109. 3263. 2513. 2445. 2875. 2682. 2610. 2966. 2803. 2711.
 2665. 3244. 2799. 2571. 3276. 3526. 2933. 2564. 2758. 2907. 3055. 3262.
 2805. 2957. 3026. 3424. 2857. 2870. 3683. 3870. 3012. 2950. 3102. 3421.
 3304. 3008. 3148. 3532. 3247. 3235. 3171. 3704. 3229. 2947. 3860. 4137.
 3445. 3196. 3191. 3356. 3578. 3667. 3346. 3349. 3562. 3830. 3355. 3429.
 3560. 3970. 3403. 3397. 4275. 4453. 3583. 3399. 3853. 3493. 3563. 3941.
 3744. 3616. 3611. 4096. 3694. 3677. 3686. 4310. 37

In [11]:
from tqdm.notebook import tqdm
#order indices roughly according to entropy/likelyhood to change under minor transformations
def pack_hash_ids(max_l):
    all_mixes = []
    for l in range(max_l):
        for j in range((l+1)//2 -1,-1,-1):
            all_mixes+=[(j,l-j),(l-j,j)]
        if l % 4 == 3:
            all_mixes+=[(l//2,l//2),(l//2+1,l//2+1)]
    return all_mixes
def pack_hash_ids2(max_l):
    all_mixes = []
    for l in range(max_l):
        for j in range((l+1)//2 -1,-1,-1):
            all_mixes+=[(j,l-j),(l-j,j)]
    return all_mixes

sym_idx128 = pack_hash_ids2(23)
nap_hash_ids24 = pack_hash_ids(24)
nap_hash_ids25 = pack_hash_ids(25)

def concat_rot_n1(dct, normi):
    dctb = [dct[i,j]*normi[idx] for idx, (i,j) in enumerate(nap_hash_ids25)]
    return np.array(dctb)
res_norm, nums_norm = [], []

for inp_vers in [orig_dct]:#, sqr_dct1]:
    thr_f = 206/256 # or 230/256
    result_weights = init_weights[:]
    #result_weights = unify_wheights(np.array(norm_sqeq0),sym_idx128)
    c0 = result_weights
    for _ in tqdm(range(2)):
        nums_ro2=[]
        for d in inp_vers:
            c = concat_rot_n1(np.abs(d.reshape(32,32)),result_weights)
            nums_ro2.append((c > thr_f*np.mean(c[:64]))*1.0) #206/256
        nums_ro2 = np.vstack(np.array(nums_ro2))
        needs_more = np.mean(np.mean(nums_ro2,axis=0)[:64]) < 0.5
        #np.mean(nums_ro2), np.min(np.mean(nums_ro2,axis=0)), np.max(np.mean(nums_ro2,axis=0)), np.mean(nums_ro2[:32])
        c0_n = np.reciprocal(np.mean(nums_ro2,axis=0))*0.5
        #c0_n[c0==c0_n] = 1.1 if needs_more else 0.9 #idea2: limiter to fixed values
        c0 = c0_n
        #c0 = unify_wheights(c0,sym_idx128)
        #result_weights = c0*result_weights
        #result_weights = np.array(result_weights)
        #result_weights[16:] = c0[16:]*result_weights[16:] #idea3: limiter to fixed indices
        #result_weights[result_weights<=1.0]=1.0 #idea4: limiter to only positive changes
        
        result_weights = np.power(c0,0.75)*result_weights
        #result_weights[274:] = np.power(c0[274:],0.75)*result_weights[274:] #idea3: limiter to fixed indices
        print(c0[16], result_weights[16], np.mean(np.mean(nums_ro2,axis=0)[:64]), np.mean(np.mean(nums_ro2,axis=0)[-64:]))
    res_norm.append(result_weights)
    nums_norm.append(nums_ro2)

0.9815407524047514 1142.9169873249534 0.49537264593051133 0.4596030504817124
0.9989461399108139 1142.0135123226744 0.49532019704433494 0.4825981407210428



In [17]:
init_weights

array([ 421.,  256.,  317.,  352.,  642.,  832.,  703.,  590.,  657.,
        603.,  992., 1051.,  659.,  744., 1059.,  909., 1159., 1353.,
       1180.,  960., 1061., 1043., 1602., 1560., 1176., 1109., 1284.,
       1556., 1484., 1347., 1695., 1823., 1616., 1207., 1340., 1448.,
       1724., 1842., 1474., 1495., 2062., 2096., 1617., 1533., 1908.,
       1751., 1720., 2079., 1893., 1568., 2179., 2364., 2036., 1744.,
       1926., 1949., 2126., 2334., 1929., 1946., 2590., 2655., 2017.,
       1917., 2163., 2430., 2355., 2155., 2170., 2593., 2349., 2162.,
       2730., 2964., 2467., 2151., 2205., 2306., 2588., 2721., 2278.,
       2469., 2564., 2902., 2307., 2334., 3109., 3263., 2513., 2445.,
       2875., 2682., 2610., 2966., 2803., 2711., 2665., 3244., 2799.,
       2571., 3276., 3526., 2933., 2564., 2758., 2907., 3055., 3262.,
       2805., 2957., 3026., 3424., 2857., 2870., 3683., 3870., 3012.,
       2950., 3102., 3421., 3304., 3008., 3148., 3532., 3247., 3235.,
       3171., 3704.,

In [19]:
res_norm[-1].astype(np.float32)

array([ 441.17386,  260.31857,  331.34088,  340.70572,  688.28357,
        848.358  ,  762.44196,  597.60406,  689.257  ,  598.43353,
       1071.2051 , 1071.1377 ,  706.7067 ,  726.98926, 1109.5193 ,
        918.51044, 1142.0135 , 1336.1497 , 1183.552  ,  988.6174 ,
       1043.4961 , 1027.9274 , 1566.6975 , 1576.1136 , 1169.3474 ,
       1118.027  , 1278.0752 , 1557.9341 , 1496.7078 , 1340.5248 ,
       1669.9738 , 1859.9269 , 1600.4939 , 1247.6624 , 1344.0001 ,
       1442.2377 , 1735.3878 , 1842.1733 , 1480.2982 , 1491.6873 ,
       2083.931  , 2144.399  , 1613.6323 , 1580.6633 , 1925.985  ,
       1744.1294 , 1729.296  , 2106.1555 , 1919.0422 , 1592.6392 ,
       2205.4834 , 2462.4263 , 2092.2437 , 1813.209  , 1935.7251 ,
       1964.1173 , 2153.2297 , 2364.2998 , 1952.857  , 1965.0398 ,
       2643.6543 , 2770.5618 , 2054.5588 , 1989.2391 , 2187.8474 ,
       2459.437  , 2401.9697 , 2184.0125 , 2223.3286 , 2653.1926 ,
       2405.993  , 2213.8357 , 2795.1165 , 3096.5974 , 2555.85

In [20]:
h_crop0.set_norm(res_norm[-1].astype(np.float32), True)
check_weights = np.ones(324,np.float32)
h_crop0.get_norm(check_weights)

In [23]:
#check difference from changing weights
check_weights-init_weights

array([ 13.,   0.,   9., -17.,  35.,   2.,  47.,  -2.,  21., -14.,  61.,
         2.,  36., -29.,  32.,  -6., -36., -39., -16.,  12., -35., -32.,
       -61., -10., -26., -10., -27., -24., -12., -29., -53.,   6., -42.,
        20., -18., -30., -17., -30., -18., -28., -13.,  13., -30.,  21.,
       -14., -36., -19.,  -8.,  -6.,  -2., -10.,  58.,  22.,  39., -22.,
       -17.,  -8.,  -9.,  -9., -14.,  10.,  70.,   3.,  39., -11., -11.,
         7.,  -7.,  16.,  16.,  17.,  15.,  19.,  81.,  46., 102., -25.,
        -4.,   4.,  -4., -10., -17.,  38.,  17.,  42.,  29.,  70., 145.,
        44., 105.,  10.,  -1.,  27.,  20.,  33.,  27.,  55.,   1.,  81.,
        80.,  83., 196., 121., 170.,  19.,  13.,  56.,  53.,  45.,  41.,
        92., 118., 107., 114., 154., 198., 114., 177.,  -1.,  49.,  74.,
        57.,  65.,  87., 108.,  60., 132., 166., 188., 160., 172., 298.,
       201., 230., -13.,  71., 101.,  88.,  79.,  77., 106., 124., 107.,
       116., 132., 200., 174., 180., 222., 350., 20

In [24]:
len(orig_dct), len(all_bu)

(114695, 114695)